In [1]:
import pickle
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
import re

# Text Vectorizer

In [2]:
def vectorize_text(text):
    """
    Esta función toma un texto como entrada, lo convierte en minúsculas y elimina las tildes
    """
    # Eliminar espacios en blanco al principio y al final del texto
    text = text.strip()
    
    # Reemplaza todos los numeros por el 0
    text = re.sub(r'\d+', '0', text)

    text = text.replace("-","")
    
    # Texto a minúscula
    text = text.lower()
    
    # Elimina la palabra acido
    text = text.replace("acido", "")
    
    return text

In [3]:
# Definir las clases objetivo
classes = ['inorganic', 'organic']

# Crear vectorizador TfidfVectorizer
# vectorizer = TfidfVectorizer(stop_words='english')
vectorizer = CountVectorizer(preprocessor=vectorize_text)

In [4]:
# Crear listas para almacenar los datos de entrenamiento y prueba
train_data = []
train_labels = []
test_data = []
test_labels = []

# Load data

In [5]:
# Leer los archivos de entrenamiento
for c in classes:
    dir_path = os.path.join('data', 'data-model', 'name-inorganic-organic', 'train', c)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        with open(file_path, 'r') as f:
            train_data.append(f.read())
            train_labels.append(c)

In [6]:
# Leer los archivos de prueba
for c in classes:
    dir_path = os.path.join('data','data-model', 'name-inorganic-organic', 'test', c)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        with open(file_path, 'r') as f:
            test_data.append(f.read())
            test_labels.append(c)

In [7]:
# Vectorizar los datos de entrenamiento y prueba
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)

# Model

In [8]:
# Entrenar el modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_vectors, train_labels)

RandomForestClassifier(random_state=42)

In [9]:
# Evaluar el modelo con los datos de prueba
accuracy = rf.score(test_vectors, test_labels)
print(f'Accuracy: {accuracy:.5f}')

Accuracy: 0.81815


# Predict

In [10]:
def predict_categories(examples):
    for input in examples:
        input_vector = vectorize_text(input)
        #print(f"Palabra original: {input} Palabra vectorizada: {input_vector}")
        # Vectorizar el texto de entrada
        input_vector = vectorizer.transform([input])

        # Realizar la predicción con el modelo de Random Forest
        prediction = rf.predict(input_vector)

        print(f'La predicción es: {prediction[0]}')


In [11]:
inorganic_names = [
    "arsano",
    "arsenito diacido de sodio",
    "hipoclorito de sodio",
    "potasiuro de boro",
    "cloruro de sodio",
    "ácido tetraoxorénico(VII)",
    "acido disulfuroso",
    "sulfurico",
    "cloroso"
]

predict_categories(inorganic_names) # 98.21%

La predicción es: inorganic
La predicción es: inorganic
La predicción es: inorganic
La predicción es: inorganic
La predicción es: inorganic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic


In [12]:
organic_names = [
    "benceno",
    "naftaleno",
    "2-cloropentanato",
    "cloruro de propilo",
    "di 2-cloropentanil éter",
    "2-bromo-2-cloropropano",
    "metanoato de isopropilo",
    "orto-difenilciclohexano",
    "2-bromo-2-cloropropil yododecil éter",
    "3-cloro-2-fluoro-hexa-1,3-dien-5-in-1-ona",
    "4-amino-2,6,6-tricloro-7,7-difluoro-89-metil-3-nitro-1,1-diyodononaconta-1,3-dien-5-ona",
]

predict_categories(organic_names)

La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic
La predicción es: organic


# Export Model

In [13]:
with open('models/rf_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [14]:
with open('models/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)